# Short hand for flexbox objects.    

In [74]:
    from ipywidgets import *
    from yaml import safe_load
    from IPython.display import *
    from pytest import fixture
    import mimetypes

    from graphviz import Source

    from mistune import markdown

Consider a list of objects that have mimetype representations.  The representations may be shorthand urls or filenames or full parameterizations

In [107]:
    _example = """- - data/IMG-0762.JPG
      - graph {"A Graph"}
      - http://roxygen.org/knuth-literate-programming.pdf
    - - text/html: http://jupyter.org
      - text/markdown: "# This is markdown"
    - - image/jpeg: 
          data: data/IMG-0762.JPG"""
    data = fixture(lambda: safe_load(_example))

`to_display` provides logic to use different IPython respresentations based off of mimetypes.

In [3]:
    def to_display(object):
        if isinstance(object, str) and object.lstrip('di').startswith('graph'):
            object = __import__('graphviz').Source(object, format='png')
        else:
            if isinstance(object, str):  
                object = {mimetypes.guess_type(object)[0] or 'text/plain': object}
            type, object = list(object.keys())[0], list(object.values())[0]
            if type.startswith('image'):
                if not isinstance(object, dict): object = {'data': object}
                object = (SVG if 'svg+xml' in type else Image)(**object)

            elif type in ('application/pdf', 'text/html'):
                if not isinstance(object, dict): 
                    object = {'src': object}
                object['width'] = object.get('width', '600px')
                object['height'] = object.get('height', '500px')
                object = IFrame(**object)
            else:
                if not isinstance(object, dict): 
                    object = {'data': object}
                object['data'] = markdown(object['data'])
                
                object = HTML(**object)

        output = Output(layout=Layout(flex='1', 
                                      min_width="200px",
                                      display="flex",
                                      flex_direction="column"))
        with output: display(object) 

        return output
    
    def display_quilt(datas):
        sliders = [
            IntSlider(value=200, min=100, max=1000, step=25, 
                      layout=Layout(width="100%")) 
            for i in "xy"]
        
        box = VBox(children=[
            Box(children=list(map(to_display, data)), 
                layout=Layout(display='flex', 
                justify_content="space-around",
                align_items="stretch",
                flex_flow="wrap"))
            for data in (datas if datas and isinstance(datas[0], list) else [datas])])
        [[
            dlink((sliders[0], 'value'), (c.layout, "min_width"), lambda x: f"{x}px"),
            dlink((sliders[1], 'value'), (c.layout, "min_height"), lambda x: f"{x}px"),
        ] for c in box.children]
        return VBox([VBox(sliders), box, VBox(sliders)])

    def display_quilt(datas):
        box = VBox(children=[
            Box(children=list(map(to_display, data)), 
                layout=Layout(display='flex', 
                justify_content="space-around",
                align_items="stretch",
                flex_flow="wrap"))
            for data in (datas if datas and isinstance(datas[0], list) else [datas])])
        return box
    
    def _demo_flexbox_quilt(data): return display_quilt(data)

In [114]:
    __name__ == '__main__' and display_quilt(data())

Design an ast node transformer to create the quilt.

In [103]:
    from deathbeds.__String_Node_Transformer import StrTokenTransformer

    class FlexBoxTransformer(StrTokenTransformer):
        condition = staticmethod(lambda str: str.startswith('- '))
        replacement = """__import__('IPython').display.display(
        __import__('importlib').import_module('deathbeds.2018-08-13-Flexbox-Transformer').display_quilt(__import__('yaml').safe_load({}))
        )"""

In [104]:
    def load_ipython_extension(ip): ip.ast_transformers.append(FlexBoxTransformer())
    def unload_ipython_extension(ip): ip.ast_transformers = [object for object in ip.ast_transformers if isinstance(object, FlexBoxTransformer)]

In [105]:
    @fixture
    def ip(): ip = __import__('IPython').get_ipython(); load_ipython_extension(ip); yield ip; unload_ipython_extension(ip)

In [106]:
    def _flexbox_string_shortcut(ip, data): ip.run_cell(f"""'''{data}'''""")